In [1]:
import os
import sys
sys.path.append(os.path.abspath('../..'))
from nowoe.data.api.smartwatch.utilities.preparations import *
import pandas
import plotly_express as px
import numpy
import json
import pickle
import gzip
import matplotlib.pyplot as plt
from copy import deepcopy
from functools import reduce
from tqdm import tqdm
from datetime import datetime, timezone
from typing import Dict, List, Tuple, Union, Any, Iterator
import torch.utils.data.dataloader
from nowoe.data.api.smartwatch.utilities.timestamp import get_utc_date_from_utc_timestamp
from nowoe.data.api.smartwatch.data_manager.module import SmartwatchDataManager
from nowoe.deep_learning.data.dataset.smartwatch_study.single_slice import get_dataloaders, SmartwatchStudySingleSliceDataset, single_slice_collate_fn
from nowoe.deep_learning.data.preprocessing.single_slice.normalization import MinMaxSingleSliceNormalization
from nowoe.deep_learning.data.tensorizer.single_slice import CustomTensorizer

In [2]:
dataloaders = get_dataloaders(
    batch_size=50,
    root_dir='../../resources/warrior_wellness/Analysis/local_repo/',
    subject_splits={
        "train": [f'SWS_{i:02d}' for i in range(0,4)],
        "test": [f'SWS_{i:02d}' for i in range(4,5)]},
    dataset_config={
        'slice_lengths': [3600],
        'slice_time_step': (5 * 60),
        'label_milestone_per_window': 1.0,
        'metadata_cache_filepath': './dataset_cache/dataset-cache-2.pkl.gz',
        'no_cache': False,
        'parallel_threads': 10
    },
    sampler_configs=dict(
       train=dict(
           negative_sample_count=100,
            positive_sample_count=50,
            target_variable='overall_quantized_stress_value',
           split_name="train"
       ),
       test=dict(
               negative_sample_count=100,
            positive_sample_count=50,
            target_variable='overall_quantized_stress_value',
           split_name="test"
       )
        )
)

2022-02-25 10:40:09,114 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - initializing data manager...
2022-02-25 10:40:51,729 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - preparing the dataset...
2022-02-25 10:40:52,028 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.dataset - INFO - Loaded metadata from cache file: /home/shayan/phoenix/nowoe_framework/nowoe_framework/notebooks/smartwatch/dataset_cache/dataset-cache-2.pkl.gz
2022-02-25 10:40:52,029 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - preparing samplers...
2022-02-25 10:40:52,101 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.sampler - WARNING - no samples for label 3.5999999999999996 was found in the split: train


In [3]:
config=dict(
            timestamp_column='utc_timestamp',
           value_config=dict(
               daily=dict(
                    embed=dict(
                        columns=['heart_rate_tsvalue'],
                        embedding_dim=[10],
                    ),
                    bring=['heart_rate_tsvalue']
               )
           )
       )
tensorizer = CustomTensorizer(
       config=config
)

In [4]:
tensorizer.learn(dataloaders['train'])

100%|███████████████████████████████████████████| 15/15 [00:05<00:00,  2.81it/s]


In [5]:
tensorizer.build_embeddings()

In [6]:
my_iter = iter(dataloaders['test'])

In [7]:
batch = next(my_iter)

In [8]:
tensorized_batch = tensorizer(batch)

In [9]:
batch['slice'][0]['daily'].shape

(240, 34)

In [10]:
tensorized_batch['slice'][0]['daily'].shape

torch.Size([240, 10])